In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
import gc
import os
%cd /content/drive/My\ Drive/BERT/jigsaw-multilingual-toxic-comment-classification/

/content/drive/My Drive/BERT/jigsaw-multilingual-toxic-comment-classification


In [5]:
train1 = pd.read_csv("data/jigsaw-toxic-comment-train.csv")
train_es = pd.read_csv('data/jigsaw-toxic-comment-train-google-es-cleaned.csv')
train_fr = pd.read_csv('data/jigsaw-toxic-comment-train-google-fr-cleaned.csv')
train_pt = pd.read_csv('data/jigsaw-toxic-comment-train-google-pt-cleaned.csv')
train_ru = pd.read_csv('data/jigsaw-toxic-comment-train-google-ru-cleaned.csv')
train_it = pd.read_csv('data/jigsaw-toxic-comment-train-google-it-cleaned.csv')
train_tr = pd.read_csv('data/jigsaw-toxic-comment-train-google-tr-cleaned.csv')

train1['lang'] = 'en'
train_es['lang'] = 'es'
train_fr['lang'] = 'fr'
train_pt['lang'] = 'pt'
train_ru['lang'] = 'ru'
train_it['lang'] = 'it'
train_tr['lang'] = 'tr'

#train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
#train2.toxic = train2.toxic.round().astype(int)
#train2['lang'] = 'en'

train = pd.concat([
    train1[['comment_text', 'lang', 'toxic']],
    train_es[['comment_text', 'lang', 'toxic']],
    train_tr[['comment_text', 'lang', 'toxic']],
    train_fr[['comment_text', 'lang', 'toxic']],
    train_pt[['comment_text', 'lang', 'toxic']],
    train_ru[['comment_text', 'lang', 'toxic']],
    train_it[['comment_text', 'lang', 'toxic']]
]).sample(n=300000).reset_index(drop=True)

del train1, train_es, train_fr, train_pt, train_ru, train_it, train_tr
gc.collect()

0

In [0]:
test = pd.read_csv('data/test.csv')
subm = pd.read_csv('data/sample_submission.csv')

In [8]:
train['comment_text'][0]


'"\n\n\xa0Soulja Boy Off In This Hoe\n\xa0Regarde-moi tourner\n\xa0Regardez-moi rouler\n\xa0Regarde-moi Crank Dat Soulja Boy\n\xa0Puis Super Man Dat Hoe\n\xa0Maintenant regarde-moi faire\n\xa0(Crank Dat Soulja Boy)\n\xa0Maintenant, regarde-moi\n\xa0(Crank Dat Soulja Boy)\n\xa0Maintenant, regarde-moi\n\xa0(Crank Dat Soulja Boy)\n\xa0Maintenant, regarde-moi\n\xa0(Crank Dat Soulja Boy)\n\n\xa0[Verset 1:]\n\xa0Soulja Boy Off In This Hoe\n\xa0Regardez-moi maigre et regardez-moi rock\n\xa0Super Man Dat Hoe\n\xa0maintenant Watch Me Crank Dat Robocop\n\xa0Super Fresh, maintenant regardez-moi Jock\n\xa0Jocking On Them Haterz mate\n\xa0Quand je fais Dat Soulja Boy\n\xa0Je penche vers la gauche et manivelle\n\xa0(Maintenant, c\'est à votre tour)\n\xa0Je Jocking On Yo xxxxx Ass\n\xa0Et si nous obtenons le Fightin\n\xa0Alors je me moque de ta chienne\n\xa0You Catch Me At Yo Local Party\n\xa0Oui, je le lance tous les jours\n\xa0Haterz Get Mad Cuz\n\xa0"" Je me suis fait quelques bapes aujourd\'hui "

In [10]:
label_cols = ['toxic']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

,toxic,none
count,300000.000000,300000.000000
mean,0.095400,0.904600
std,0.293767,0.293767
min,0.000000,0.000000
25%,0.000000,1.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [0]:
train['comment_text'].fillna("unknown", inplace=True)
test['content'].fillna("unknown", inplace=True)

In [0]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [0]:
n = train.shape[0]

vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

trn_term_doc = vec.fit_transform(train['comment_text'])
test_term_doc = vec.transform(test['content'])

In [14]:
trn_term_doc, test_term_doc


(<300000x5961846 sparse matrix of type '<class 'numpy.float64'>'
 	with 36500351 stored elements in Compressed Sparse Row format>,
 <63812x5961846 sparse matrix of type '<class 'numpy.float64'>'
 	with 5782767 stored elements in Compressed Sparse Row format>)

In [0]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [0]:
x = trn_term_doc
test_x = test_term_doc

In [0]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=False)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [18]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit toxic


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)

submission.to_csv('submission_nb.csv', index=False)